In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings('always')
from sklearn.model_selection import train_test_split
import shapely.speedups
from sklearn.preprocessing import StandardScaler
from IPython.display import Image
import re
from itertools import combinations
import random
import joblib

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.preprocessing import StandardScaler

import tensorflow as tf
from tensorflow import keras
import keras_tuner as kt
#
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
def plot_regression(modelo,x,y,title="",xlabel="x",ylabel="y"):
    plt.figure()
    
    plt.plot(x,y,"o",label="Valores verdaderos")
    plt.plot(x,modelo.predict(x),"x",label="Valores estimados")
    
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.legend()
    plt.show()

In [ ]:
viviendas_train=pd.read_csv('./TP1/Datasets/viviendas_caba_train.csv').copy()
viviendas_test=pd.read_csv('./TP1/Datasets/viviendas_caba_test.csv').copy()
viviendas = pd.concat([viviendas_train, viviendas_test])

# regresion

In [ ]:
Realizamos un one hot encoding para poder utilizar el barrio y type para regresion

In [ ]:
viviendas = pd.get_dummies(viviendas, columns = ["property_type"])
viviendas = pd.get_dummies(viviendas, columns = ["barrio"])

In [ ]:
definimos las columnas que utilizaremos para entrenar las redes

In [ ]:
columnas_predictoras=viviendas.columns.to_list()
for variable in ['property_price', 'start_date', 'end_date', 'property_title', 'geometry']:
    columnas_predictoras.remove(variable)
columnas_predictoras

## RANDOMIZADOR


In [ ]:
Para ontrenar las redes neuronales utilizaresmos un random_search para optimizar los hiperparametros

In [ ]:
#al instanciar un ModeloRedes crea y entrena una red neuronal con los hiperparametros pasados
class ModeloRedes:
    def __init__(self, dataset, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size, cantidad_filas):
        self.variables_predictoras = variables_predictoras
        self.cantidad_primera_capa = cantidad_de_primera_capa
        self.funcion_activacion = funcion_activacion
        self.optimizador = optimizador
        self.learning_rate = learning_rate
        self.resultado = None
        self.metrica = metrica
        self.epoch = epoch
        self.batch_size = batch_size
        self.modelo = None

        
        self.x_test = None
        self.y_test = None
        self.x_train = None
        self.y_train = None
        self.dataset_trabajo = dataset[:cantidad_filas]
        self.obtener_datasets()
        self.entrenar_modelo()

    def obtener_datasets(self):    
        self.x_train, self.x_test, self.y_train, self.y_test = train_test_split(self.dataset_trabajo.loc[:,self.variables_predictoras],self.dataset_trabajo.loc[:,'property_price'],test_size=0.2)
        self.escalar_datasets()
    
    def escalar_datasets(self):
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(self.x_train))
        self.x_train=sscaler.transform(pd.DataFrame(self.x_train))
        self.x_test=sscaler.transform(pd.DataFrame(self.x_test))

    def entrenar_modelo(self):
    # Creo un modelo Red Neuronal
        d_in= len(self.variables_predictoras)
        d_out=1

        capas = [
            # input layer
            keras.layers.Dense(d_in,input_shape=(d_in,), activation= self.funcion_activacion)
        ]

        #inserta capas que van decreciendo en cantidad de neuronas
        for i in range(self.cantidad_primera_capa,2,-1):
            capas.append(keras.layers.Dense(i, activation= self.funcion_activacion ))

        capas.append(keras.layers.Dense(d_out, ))
        self.modelo = keras.Sequential(capas)

        self.modelo.compile(
        optimizer= self.optimizador(self.learning_rate), 
        loss=self.metrica, 
        metrics=[self.metrica], 
        )

        self.modelo.fit(self.x_train,self.y_train,epochs=self.epoch,batch_size=self.batch_size,verbose=False)
    
    def plotear(self):
        plot_regression(self.modelo,self.x_test,self.y_test,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        
    def testear_modelo(self):
        y_pred=self.modelo.predict(self.x_test)
        mse=mean_squared_error(self.y_test,y_pred)
        self.resultado = mse ** (1/2)
        return self.resultado
    
    def predecir(self, df):

        df_para_predecir =  df.loc[:,self.variables_predictoras]
        df_precios = df.loc[:,'property_price']
        
        sscaler=StandardScaler()
        sscaler.fit(pd.DataFrame(df_para_predecir))
        df_para_predecir=sscaler.transform(pd.DataFrame(df_para_predecir))

        
        precios_pred=self.modelo.predict(df_para_predecir)
        mse=mean_squared_error(df_precios,precios_pred)
        resultado = mse ** (1/2)
        #plot_regression(self.modelo,df_para_predecir,df_precios,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")
        return resultado
    
    def info(self):
        print('\n','\n')
        print('\n',
        self.variables_predictoras , '\n',
        self.cantidad_primera_capa ,'\n',
        self.funcion_activacion ,'\n',
        self.optimizador ,'\n',
        self.learning_rate ,'\n',
        self.metrica ,'\n',
        self.epoch ,'\n',
        self.batch_size ,'\n',
        self.dataset_trabajo.shape ,'\n')

In [ ]:
columnas_importantes =  ['latitud', 'longitud', 'property_surface_total', 'property_surface_covered']
columnas_extras = []
for e in columnas_predictoras:
    if e not in columnas_importantes:
        columnas_extras.append(e)

print(columnas_extras)        

def agregar_columnas(posibles_variables_predictoras, columnas, cant_columnas):
            i = 0
            while (i < cant_columnas): 
                pos = random.randrange(i,len(columnas), 1)
                posibles_variables_predictoras.append(columnas[pos])
                columnas[i], columnas[pos] = columnas[pos], columnas[i]
                i += 1

posibles_cantidad_de_primera_capa = []
for i in range(1,20):
    posibles_cantidad_de_primera_capa.append(i)

posibles_funcion_activacion = ["relu", "tanh", "softmax", "sigmoid"]
posibles_metricas = ['mse', 'mae']

posibles_learning_rate = [0.1, 0.01]

In [ ]:
def funcion_optimizadoraNADAM(learnigRate):
            return keras.optimizers.Nadam(learning_rate = learnigRate)

def funcion_optimizadoraRMSPROP(learnigRate):
            return keras.optimizers.RMSprop(learning_rate = learnigRate)

def funcion_optimizadoraADAMAX(learnigRate):
            return keras.optimizers.Adamax(learning_rate = learnigRate)

def funcion_optimizadoraADADELTA(learnigRate):
            return keras.optimizers.Adadelta(learning_rate = learnigRate)

posibles_optimizador = [funcion_optimizadoraNADAM, funcion_optimizadoraRMSPROP, funcion_optimizadoraADAMAX, funcion_optimizadoraADADELTA]

In [ ]:
def obtener_posibles_variables_predictoras():
        posibles_variables_predictoras = []     
        cant_columnas_importantes = random.randrange(1,len(columnas_importantes)+1, 1)
        cant_columnas_extras = random.randrange(0,len(columnas_extras)+1, 1)

        agregar_columnas(posibles_variables_predictoras, columnas_importantes, cant_columnas_importantes)
        agregar_columnas(posibles_variables_predictoras, columnas_extras, cant_columnas_extras)
        return posibles_variables_predictoras

In [ ]:
def random_validator(cantidad_modelos, columnas_importantes, columnas_extras, posibles_cantidad_de_primera_capa, 
    posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas, obtener_posibles_variables_predictoras):
    resultados = []
    for i in range(cantidad_modelos):

        variables_predictoras = obtener_posibles_variables_predictoras()
        cantidad_de_primera_capa = random.choice(posibles_cantidad_de_primera_capa)
        funcion_activacion = random.choice(posibles_funcion_activacion)
        optimizador = random.choice(posibles_optimizador)
        learning_rate = random.choice(posibles_learning_rate)
        metrica = random.choice(posibles_metricas)
        epoch = random.randrange(100, 300, 10)
        batch_size = random.randrange(10, 1000, 1)
        cantidad_filas = 3000#random.randrange(3000, 5000, 1)

        print('\n','\n')
        print(i , '\n',
        variables_predictoras , '\n',
        cantidad_de_primera_capa ,'\n',
        funcion_activacion ,'\n',
        optimizador ,'\n',
        learning_rate ,'\n',
        metrica ,'\n',
        epoch ,'\n',
        batch_size ,'\n',
        cantidad_filas ,'\n')

        modelo = ModeloRedes(viviendas, variables_predictoras, cantidad_de_primera_capa, funcion_activacion, optimizador, learning_rate, metrica, epoch, batch_size, cantidad_filas)
        resultados.append([modelo, modelo.testear_modelo()])
        print(modelo.testear_modelo())
    return resultados

In [ ]:
def func():
    resultados = random_validator(50,columnas_importantes, columnas_extras, posibles_cantidad_de_primera_capa, 
        posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas,obtener_posibles_variables_predictoras)

    return resultados

In [ ]:
def reentrenar(modelo):
    modelo.info()
    modelo.testear_modelo()
    mejor_modelo = modelo
    mejor_resultado = modelo.resultado 
   
    for i in range(5):
        print('valor a vencer', mejor_resultado)
        modelo_aux = ModeloRedes(viviendas, modelo.variables_predictoras, modelo.cantidad_primera_capa, modelo.funcion_activacion, modelo.optimizador, modelo.learning_rate, modelo.metrica, modelo.epoch, modelo.batch_size, 3000)
        resultado_aux = modelo_aux.testear_modelo()
        print('reentrenando intento', i+1)
        resultado_aux = modelo_aux.testear_modelo()
        print(resultado_aux)
        modelo.info()
        
        if resultado_aux < mejor_resultado:
            mejor_resultado = resultado_aux
            mejor_modelo = modelo_aux 
    return [mejor_modelo, mejor_resultado]

In [ ]:
def obtener_mejores(resultados):
    mejores = []
    for i in resultados:
        if (i[1] < 130000):
            mejores.append(i)

    """for i in range(len(mejores)):
        mejores[i] = reentrenar(mejores[i][0])"""
    return mejores

In [ ]:
mejores = []

Hacemos un random search de 50 redes para ver que patrones dan mejores resultados

In [ ]:
random_validator(50,columnas_importantes, columnas_extras, posibles_cantidad_de_primera_capa, 
        posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas,obtener_posibles_variables_predictoras)

Despues de la primera iteracion pudimos observar que todos los buenos modelos eran de relu, por lo que lo ponemos como parametro fijo

In [ ]:
posibles_funcion_activacion = ["relu"]

resultados = random_validator(50,columnas_importantes, columnas_extras, posibles_cantidad_de_primera_capa, 
        posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas)
mejores.append(obtener_mejores(resultados))

Despues de la segunda iteracion vemos que todos los modelos contienen muchas variables_predictoras extras

In [ ]:
def obtener_posibles_variables_predictoras():
        posibles_variables_predictoras = []     
        cant_columnas_importantes = random.randrange(1,len(columnas_importantes)+1, 1)
        cant_columnas_extras = random.randrange(30,len(columnas_extras)+1, 1)

        agregar_columnas(posibles_variables_predictoras, columnas_importantes, cant_columnas_importantes)
        agregar_columnas(posibles_variables_predictoras, columnas_extras, cant_columnas_extras)
        return posibles_variables_predictoras

In [ ]:
resultados = random_validator(50,columnas_importantes, columnas_extras, posibles_cantidad_de_primera_capa, 
        posibles_funcion_activacion, posibles_optimizador, posibles_learning_rate, posibles_metricas,obtener_posibles_variables_predictoras)
mejores.append(obtener_mejores(resultados))

In [ ]:
for i in mejores:
    for j in i:
        print(j)
        j[0].info()

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:

resultados = func()
mejores.append(obtener_mejores(resultados))

In [ ]:
for i in mejores:
    for j in i:
        print(j)
        #j[0].info()

In [ ]:
modelo.info()

In [ ]:
modelo = mejores[len(mejores)-1][0][0]

model_tot = ModeloRedes(
        viviendas, 
        modelo.variables_predictoras, 
        modelo.cantidad_primera_capa, 
        modelo.funcion_activacion, 
        modelo.optimizador, 
        modelo.learning_rate, 
        modelo.metrica, 
        modelo.300, 
        modelo.batch_size, 
        viviendas.size)
a = model_tot.testear_modelo()
a

In [ ]:
def reentrenar_tot(modelo):
    modelo.info()
    mejor_modelo = ModeloRedes(viviendas, modelo.variables_predictoras, modelo.cantidad_primera_capa, modelo.funcion_activacion, modelo.optimizador, modelo.learning_rate, modelo.metrica, modelo.epoch, modelo.batch_size, viviendas.size)
    mejor_modelo.testear_modelo()
    mejor_resultado = mejor_modelo.resultado 
   
    if mejor_resultado > 150000:
        return None

    for i in range(0):
        print('valor a vencer', mejor_resultado)
        modelo_aux = ModeloRedes(viviendas, modelo.variables_predictoras, modelo.cantidad_primera_capa, modelo.funcion_activacion, modelo.optimizador, modelo.learning_rate, modelo.metrica, modelo.epoch, modelo.batch_size, viviendas.size)
        resultado_aux = modelo_aux.testear_modelo()
        print('reentrenando intento', i+1)
        resultado_aux = modelo_aux.testear_modelo()
        print(resultado_aux)
        modelo_aux.info()
        
        if resultado_aux < mejor_resultado:
            mejor_resultado = resultado_aux
            mejor_modelo = modelo_aux 
    return [mejor_modelo, mejor_resultado]

In [112]:
tots = []
for i in mejores:
    for j in i:
        print(j)
        tots.append(reentrenar_tot(j[0]))
        

[<__main__.ModeloRedes object at 0x7f4b7028ac10>, 108139.9216440872]

 


 ['longitud', 'latitud', 'property_surface_total', 'property_surface_covered', 'barrio_Palermo', 'barrio_Saavedra', 'barrio_Parque Avellaneda', 'property_rooms', 'barrio_Belgrano', 'barrio_San Cristobal', 'barrio_Almagro', 'barrio_Floresta', 'barrio_Nuñez', 'barrio_Flores', 'barrio_Villa Lugano', 'barrio_Pompeya', 'property_bedrooms', 'barrio_Parque Centenario', 'barrio_Villa Crespo', 'property_type_PH', 'barrio_Barracas', 'barrio_Villa Soldati', 'barrio_Parque Chas', 'barrio_Boca', 'barrio_Abasto', 'barrio_Constitución', 'barrio_Tribunales', 'barrio_Parque Chacabuco', 'barrio_Catalinas', 'property_type_Departamento', 'barrio_Velez Sarsfield', 'barrio_Villa Riachuelo', 'barrio_Villa Luro', 'barrio_Barrio Norte', 'barrio_Monte Castro', 'barrio_Villa Urquiza', 'barrio_Agronomía', 'barrio_Monserrat', 'barrio_Balvanera', 'barrio_Parque Patricios', 'barrio_Mataderos', 'barrio_Colegiales', 'barrio_Centro / Microcentro'

/home/pal/.local/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.bool) or is_bool_dtype(data):
/home/pal/.local/lib/python3.8/site-packages/pandas/core/indexes/base.py:395: DeprecationWarning: `np.bool` is a deprecated alias for the builtin `bool`. To silence this warning, use `bool` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.bool_` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  elif issubclass(data.dtype.type, np.

KeyboardInterrupt: 

In [111]:
print(tots)
print('1')

[[<__main__.ModeloRedes object at 0x7f4b6eaa71c0>, 109827.51856672103], [<__main__.ModeloRedes object at 0x7f4b6e0c3ee0>, 130705.42420404805], [<__main__.ModeloRedes object at 0x7f4b6dc3ca90>, 145723.08073309163]]
1


In [ ]:
joblib.dump(model_tot, './Models/red_regresion_108k_tot') 

In [ ]:
for i in mejores:
    for j in i:
        print('El modelo tiene metrica')
        print(j[1])
        j[0].info()
        j[0].imprimir()
        print('El modelo predice sobre todo el dataset como')
        j.append(j[0].predecir(viviendas))
        print(j[2])


In [ ]:
"""va = ['longitud', 'property_surface_total', 'property_surface_covered', 'latitud', 'barrio_Paternal', 'barrio_Retiro', 'barrio_Las Cañitas', 'barrio_Pompeya', 'barrio_Balvanera', 'barrio_Monserrat', 'barrio_Villa Pueyrredón', 'barrio_Mataderos', 'barrio_Chacarita', 'barrio_Barracas', 'property_bedrooms', 'barrio_San Nicolás', 'barrio_Villa General Mitre', 'barrio_Liniers', 'barrio_Villa Santa Rita', 'barrio_San Telmo', 'property_type_Departamento', 'barrio_Villa Lugano', 'barrio_Almagro', 'barrio_Coghlan', 'barrio_Once', 'barrio_Villa Soldati', 'barrio_Villa Ortuzar', 'barrio_Floresta', 'barrio_Villa Riachuelo', 'barrio_Constitución', 'barrio_Congreso', 'barrio_Velez Sarsfield', 'barrio_Villa del Parque', 'barrio_Villa Devoto', 'barrio_Abasto', 'barrio_Villa Urquiza', 'barrio_Catalinas', 'barrio_Recoleta', 'barrio_Caballito', 'barrio_Nuñez', 'barrio_Barrio Norte', 'barrio_Versalles', 'barrio_Monte Castro', 'barrio_Boedo', 'barrio_Belgrano', 'barrio_Boca', 'barrio_Parque Avellaneda', 'barrio_Tribunales', 'barrio_Villa Real', 'barrio_Villa Crespo', 'barrio_Centro / Microcentro', 'property_rooms', 'barrio_Parque Patricios', 'property_type_PH', 'barrio_Agronomía', 'barrio_Colegiales', 'barrio_Flores', 'barrio_Saavedra', 'barrio_San Cristobal', 'property_type_Casa', 'barrio_Puerto Madero', 'barrio_Parque Centenario'] 

modelo = ModeloRedes(viviendas, va, 15, 'relu', funcion_optimizadoraADADELTA, 0.01, 'mae', 200, 125, 2000)
print(modelo.testear_modelo())
modelo.info()
modelo.imprimir()"""

In [ ]:
"""modelo = mejores[0][0][0]

df_para_predecir =  viviendas.loc[:,modelo.variables_predictoras]
df_precios = viviendas.loc[:,'property_price']

sscaler=StandardScaler()
sscaler.fit(pd.DataFrame(df_para_predecir))
df_para_predecir=sscaler.transform(pd.DataFrame(df_para_predecir))


precios_pred=modelo.modelo.predict(df_para_predecir)
mse=mean_squared_error(df_precios,precios_pred)
print (mse ** (1/2))
plot_regression(modelo.modelo,df_para_predecir,df_precios,title=f"Modelo Final",xlabel="Horsepower",ylabel="MPG")"""

In [ ]:
#joblib.dump(mejores[0][0][0], './Models/red_regresion_67k') 